# SparkSQL and MLib Assignment

## Part A: SparkSQL

#### Course: Big Data Systems and Architectures           Master: Business Analytics - Full Time
#### Student Name: Lakkas - Pyknis Evangelos              Student ID:f2822306     email: eva.lakkaspyknis@aueb.gr  

In [61]:
from pyspark.sql import SparkSession

In [62]:
from pyspark import SparkContext, SparkConf
import pandas as pd

In [63]:
spark = SparkSession.builder \
    .appName("IMDB Analysis") \
    .getOrCreate()


In [64]:
imdb_df = spark.read.json("movie.json")

In [65]:
imdb_df.createOrReplaceTempView("imdb_data")

In [8]:
spark.sql("SELECT * FROM imdb_data").show()

+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+-------+--------------------+--------------------+------------+---------+----+
|_corrupt_record|              actors|           countries|         description|           directors|               genre|            imdb_url|             img_url|           languages|metascore|rating|runtime|             tagline|               title|users_rating|    votes|year|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+-------+--------------------+--------------------+------------+---------+----+
|              [|                null|                null|                null|                null|                null|                null|              

In [66]:
pandas_imdb = imdb_df.toPandas()


In [11]:
pandas_imdb.head(8)

,_corrupt_record,actors,countries,description,directors,genre,imdb_url,img_url,languages,metascore,rating,runtime,tagline,title,users_rating,votes,year
0,[,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,"[Timothée Chalamet, Elle Fanning, Liev Schreib...",[USA],A young couple arrives in New York for a weeke...,[Woody Allen],"[Comedy, Romance]",https://www.imdb.com/title/tt7139936/,https://m.media-amazon.com/images/M/MV5BODAwZD...,[English],44,PG-13,92 min,None,A Rainy Day in New York,6.6,"21,903",2019
2,None,"[Emilia Clarke, Hadley Fraser, Sylvia Panacion...",[USA],"Creepy, terrifying chapters from our book of h...","[Michael Escobedo, Kelly Hallmark, 4 more cred...","[Horror, Thriller]",https://www.imdb.com/title/tt12384178/,https://m.media-amazon.com/images/M/MV5BMDI5ZD...,[English],None,18,91 min,"When it comes to murder, they wrote the book",Murder Manual,2.4,192,2020
3,None,"[Matthew Broderick, Alan Ruck, Mia Sara, Jeffr...",[USA],A high school wise guy is determined to have a...,[John Hughes],[Comedy],https://www.imdb.com/title/tt0091042/,https://m.media-amazon.com/images/M/MV5BMDA0Nj...,"[English, German]",61,PG-13,103 min,Because life is too beautiful a thing to waste.,Ferris Bueller's Day Off,7.8,"308,847",1986
4,None,"[Robert De Niro, Nick Nolte, Jessica Lange, Ju...",[USA],"A convicted rapist, released from prison after...",[Martin Scorsese],"[Crime, Thriller]",https://www.imdb.com/title/tt0101540/,https://m.media-amazon.com/images/M/MV5BNjdhNz...,[English],73,R,128 min,There is nothing in the dark that isn't there ...,Cape Fear,7.3,"165,628",1991
5,None,"[Lindsay Lohan, Rachel McAdams, Tina Fey, Tim ...","[USA, Canada]","Cady Heron is a hit with The Plastics, the A-l...",[Mark Waters],[Comedy],https://www.imdb.com/title/tt0377092/,https://m.media-amazon.com/images/M/MV5BMjE1MD...,"[English, German, Vietnamese, Swahili]",66,PG-13,97 min,Survival of the Ruthless,Mean Girls,7.0,"320,492",2004
6,None,"[Sophia Lillis, Samuel Leakey, Alice Krige, Je...","[Canada, Ireland, USA, South Africa]",A long time ago in a distant fairy tale countr...,[Oz Perkins],"[Fantasy, Horror, Mystery, Thriller]",https://www.imdb.com/title/tt9086228/,https://m.media-amazon.com/images/M/MV5BOTIyYW...,[English],64,PG-13,87 min,Follow the crumbs.,Gretel & Hansel,5.3,"15,051",2020
7,None,"[Chris Pratt, Bryce Dallas Howard, Rafe Spall,...",[USA],When the island's dormant volcano begins roari...,[J.A. Bayona],"[Action, Adventure, Sci-Fi]",https://www.imdb.com/title/tt4881806/,https://m.media-amazon.com/images/M/MV5BNzIxMj...,"[English, Russian]",51,PG-13,128 min,Off the Island. Into your Home.,Jurassic World: Fallen Kingdom,6.2,"250,190",2018


## Task A1
*counting the number of movies/records in the database*

In [68]:
movie_count = spark.sql("SELECT COUNT(*) AS movie_count FROM imdb_data").collect()[0]['movie_count']

In [69]:
print("Number of movies in the database:", movie_count)

Number of movies in the database: 62060


*counting PG-13 movies*

In [70]:
PG13_count = spark.sql("SELECT COUNT(*) AS PG13_count FROM imdb_data WHERE rating='PG-13' ").collect()[0]['PG13_count']

In [71]:
print("Number of PG-13 rated movies in the database:", PG13_count)

Number of PG-13 rated movies in the database: 3771


*votes statistics*

In [72]:
votes_summary = imdb_df.select("votes").describe()

In [73]:
votes_summary.show()

+-------+------------------+
|summary|             votes|
+-------+------------------+
|  count|             62056|
|   mean|161.04686499637202|
| stddev|218.14851861026264|
|    min|             1,000|
|    max|           999,637|
+-------+------------------+



*counting the distinct values of genres column in descending order*

I wrote a sparkSQL query that selects the genre column from the dataset and counts the occurences of the values of each genre based on a GROUP BY function
and it sorts the countings from the most common one to te least

In [74]:
genre_counts = spark.sql("SELECT genre, COUNT(*) AS genre_count FROM imdb_data GROUP BY genre ORDER BY genre_count desc")

In [75]:
genre_counts.show()

+--------------------+-----------+
|               genre|genre_count|
+--------------------+-----------+
|             [Drama]|       8037|
|            [Comedy]|       5153|
|            [Horror]|       2677|
|     [Comedy, Drama]|       2090|
|    [Drama, Romance]|       1604|
|   [Comedy, Romance]|       1497|
|[Comedy, Drama, R...|       1341|
|          [Thriller]|       1338|
|                  []|       1252|
|           [Western]|       1171|
|  [Horror, Thriller]|       1045|
|      [Crime, Drama]|        937|
|   [Drama, Thriller]|        861|
|            [Action]|        733|
|[Crime, Drama, Th...|        639|
|    [Comedy, Horror]|        629|
|     [Action, Drama]|        488|
|         [Adventure]|        405|
|[Horror, Mystery,...|        398|
|            [Sci-Fi]|        355|
+--------------------+-----------+
only showing top 20 rows



In [39]:
spark.stop()